In [ ]:
import numpy as np
import time
import plotly.io as pio
from pyVHR import extraction
from pyVHR import datasets
import plotly.graph_objects as go

In [ ]:
pio.renderers.default = 'browser'

In [ ]:
'''
 import os

# 递归遍历文件夹及其子文件夹下的所有txt文件
def find_txt_files(folder_path):
    txt_files = []

    # 遍历文件夹中的所有文件和子文件夹
    for root, dirs, files in os.walk(folder_path):
        # 遍历当前文件夹中的所有文件
        for file in files:
            # 检查文件扩展名是否为txt
            if file.endswith('.txt'):
                # 构建txt文件的完整路径
                txt_file_path = os.path.join(root, file)
                if 'Lab' in txt_file_path[-10:]:
                # 将txt文件路径添加到列表中
                    txt_files.append(txt_file_path)

    return txt_files

# 指定文件夹路径
folder_path = r"F:\Data Set\pyVHR\ECG_FITNESS Signal_good_video"

# 调用函数查找文件夹及其子文件夹下的所有txt文件
txt_files = find_txt_files(folder_path)

# 遍历输出找到的txt文件路径
for file in txt_files:
    print(file)
print(len(txt_files))

for file in txt_files:
    try:
        os.remove(file)
        print(f"文件 {file} 已成功删除")
    except OSError as e:
        print(f"删除文件 {file} 异常: {e}")
'''

In [ ]:
dataset_name = 'ecg_fitness'                              # 数据集名称
video_DIR = r"F:\Data Set\pyVHR\ECG_FITNESS"         # 视频数据目录的路径
BVP_DIR = r"F:\Data Set\pyVHR\ECG_FITNESS"             # 相应的BVP数据目录的路径

w_size = 10                # 每次处理视频的滑窗长度 ，w_size=视频时长，则timesGT的第一个值对应的bpmGT中的心率就是对应整个视频时长的平均心率
overlap = 1                # 相邻滑窗在时间上的重叠长度
video_idx = 0              # 待处理视频的索引
seconds = 0                # 处理视频的时长
scal = 1.0                 # 视频尺寸缩放比例
landmarks = extraction.MagicLandmarks.cheek_left_top \
            + extraction.MagicLandmarks.cheek_right_top   # 选择Landmark点

dataset = datasets.datasetFactory(dataset_name, videodataDIR=video_DIR, BVPdataDIR=BVP_DIR)  # 读取对应数据集的类方法
fname = dataset.getSigFilename(video_idx)     # 读取索引为video_idx的视频，这里的范围与上述的all_video有关
sigGT = dataset.readSigfile(fname)            # 读取视频对应的BVP.gt数据
test_bvp = sigGT.data
bpmGT, timesGT = sigGT.getBPM(winsize=w_size)         # GT信息

videoFileName = dataset.getVideoFilename(video_idx)
fps = int(extraction.get_fps(videoFileName))

# 定义信号提取方法
sig_extractor = extraction.SignalProcessing()  # 创建一个可选用不同方法的用于提取信号的类
sig_extractor.set_skin_extractor(extraction.SkinExtractionConvexHull())  # 设置提取人脸皮肤的方法，ConvexHull法
sig_extractor.set_total_frames(seconds * fps)  # 得到需要处理的视频的帧数，seconds=0处理所有帧
sig_extractor.set_landmarks(landmarks)
sig_extractor.set_visualize_skin_and_landmarks(visualize_skin=True, visualize_landmarks=True,
                                               visualize_landmarks_number=False, visualize_patch=True)  # 可视化参数设置
sig_extractor.set_square_patches_side(10.0)  # 设置用于patches提取的矩形框尺寸

print('All Videos: ', dataset.numVideos)
print('Video processed name: ', videoFileName)
print('Video frame rate:     ', fps)

In [ ]:
dataset_name = 'ecg_fitness'                              # 数据集名称
video_DIR = r"F:\Data Set\pyVHR\ECG_FITNESS"           # 视频数据目录的路径
BVP_DIR = r"F:\Data Set\pyVHR\ECG_FITNESS"             # 相应的BVP数据目录的路径

seconds = 0                # 处理视频的时长
landmarks = extraction.MagicLandmarks.cheek_left_top + extraction.MagicLandmarks.cheek_right_top   # 选择Landmark点
# landmarks = [100, 119, 118, 117, 50, 36, 205, 207, 206, 348, 346, 347, 330, 280, 425, 426, 427, 411]

sig_extractor = extraction.SignalProcessing()  # 创建一个可选用不同方法的用于提取信号的类
sig_extractor.set_skin_extractor(extraction.SkinExtractionConvexHull())  # 设置提取人脸皮肤的方法，ConvexHull法
sig_extractor.set_landmarks(landmarks)
sig_extractor.set_visualize_skin_and_landmarks(visualize_skin=True, visualize_landmarks=True,
                                               visualize_landmarks_number=False, visualize_patch=True)  # 可视化参数设置
sig_extractor.set_square_patches_side(25.0)  # 设置用于patches提取的矩形框尺寸


dataset = datasets.datasetFactory(dataset_name, videodataDIR=video_DIR, BVPdataDIR=BVP_DIR)  # 读取对应数据集的类方法
count = dataset.numVideos
print('All Videos: ', count)
start_time0 = time.time()

# for index in range(count):
for index in range(1):
    fname = dataset.getSigFilename(index)                        # 读取索引为video_idx的视频，这里的范围与上述的all_video有关
    videoFileName = dataset.getVideoFilename(index)
    fps = int(extraction.get_fps(videoFileName))
    sig_extractor.set_total_frames(seconds * fps)                 # 得到需要处理的视频的帧数，seconds=0处理所有帧
    print(index, ': ', videoFileName)
    start_time1 = time.time()
    # raw_rgb, motion_sig = sig_extractor.extract_patches(videoFileName, "squares", "mean")  # Patches方法
    raw_rgb, motion_sig = sig_extractor.extract_holistic(videoFileName)                      # Holistic方法

    # visualize_coll = sig_extractor.get_visualize_skin()           # patches可视化图像信号
    # skin_area = []
    # for skin in visualize_coll:
    #     matrix_one = np.ones((skin.shape[0], skin.shape[1]))
    #     matrix_and = np.logical_and(matrix_one, skin[:,:,0])
    #     skin_area.append(sum(sum(matrix_and)))
    # skin_area = np.array(skin_area)

    # 将提取到的RGB信息与运动信息分别存储
    txt_R_file = videoFileName[:-4] + '-R.txt'
    txt_G_file = videoFileName[:-4] + '-G.txt'
    txt_B_file = videoFileName[:-4] + '-B.txt'
    np.savetxt(txt_R_file, raw_rgb[:, :, 0])
    np.savetxt(txt_G_file, raw_rgb[:, :, 1])
    np.savetxt(txt_B_file, raw_rgb[:, :, 2])

    # txt_patches_file = videoFileName[:-4] + '-patch_index.txt'
    # txt_X_file = videoFileName[:-4] + '-X_Motion.txt'
    # txt_Y_file = videoFileName[:-4] + '-Y_Motion.txt'
    # txt_Z_file = videoFileName[:-4] + '-Z_Motion.txt'

    # np.savetxt(txt_patches_file, motion_sig[:, :, 0])
    # np.savetxt(txt_X_file, motion_sig[:, :, 1])
    # np.savetxt(txt_Y_file, motion_sig[:, :, 2])
    # np.savetxt(txt_Z_file, skin_area)

    end_time1 = time.time()
    print('Get RGB signal time used :', end_time1 - start_time1)
    print(' ')

end_time0 = time.time()
print('All time used :', end_time0 - start_time0)


In [ ]:
dataset_name = 'ecg_fitness'                              # 数据集名称
video_DIR = r"E:\Data Set\pyVHR\ECG_FITNESS"           # 视频数据目录的路径
BVP_DIR = r"E:\Data Set\pyVHR\ECG_FITNESS"             # 相应的BVP数据目录的路径

seconds = 0                # 处理视频的时长
landmarks = extraction.MagicLandmarks.cheek_left_top + extraction.MagicLandmarks.cheek_right_top   # 选择Landmark点

sig_extractor = extraction.SignalProcessing()  # 创建一个可选用不同方法的用于提取信号的类
sig_extractor.set_skin_extractor(extraction.SkinExtractionConvexHull())  # 设置提取人脸皮肤的方法，ConvexHull法
sig_extractor.set_landmarks(landmarks)
sig_extractor.set_visualize_skin_and_landmarks(visualize_skin=True, visualize_landmarks=True,
                                               visualize_landmarks_number=False, visualize_patch=True)  # 可视化参数设置
sig_extractor.set_square_patches_side(10.0)  # 设置用于patches提取的矩形框尺寸


dataset = datasets.datasetFactory(dataset_name, videodataDIR=video_DIR, BVPdataDIR=BVP_DIR)  # 读取对应数据集的类方法
count = dataset.numVideos
print('All Videos: ', count)
start_time0 = time.time()

for index in range(count):
    fname = dataset.getSigFilename(index)                        # 读取索引为video_idx的视频，这里的范围与上述的all_video有关
    videoFileName = dataset.getVideoFilename(index)
    fps = int(extraction.get_fps(videoFileName))
    sig_extractor.set_total_frames(seconds * fps)                 # 得到需要处理的视频的帧数，seconds=0处理所有帧
    print(index, ': ', videoFileName)
    start_time1 = time.time()
    raw_rgb, motion_sig = sig_extractor.extract_patches(videoFileName, "squares", "mean")  # Patches方法
    # raw_rgb, motion_sig = sig_extractor.extract_holistic(videoFileName)                      # Holistic方法

    visualize_coll = sig_extractor.get_visualize_skin()           # patches可视化图像信号
    skin_area = []
    for skin in visualize_coll:
        matrix_one = np.ones((skin.shape[0], skin.shape[1]))
        matrix_and = np.logical_and(matrix_one, skin[:,:,0])
        skin_area.append(sum(sum(matrix_and)))
    skin_area = np.array(skin_area)

    # 将提取到的RGB信息与运动信息分别存储
    txt_R_file = videoFileName[:-4] + '-R.txt'
    txt_G_file = videoFileName[:-4] + '-G.txt'
    txt_B_file = videoFileName[:-4] + '-B.txt'
    np.savetxt(txt_R_file, raw_rgb[:, :, 0])
    np.savetxt(txt_G_file, raw_rgb[:, :, 1])
    np.savetxt(txt_B_file, raw_rgb[:, :, 2])

    end_time1 = time.time()
    print('Get RGB signal time used :', end_time1 - start_time1)
    print(' ')

end_time0 = time.time()
print('All time used :', end_time0 - start_time0)

In [ ]:
r = np.loadtxt("E:\\Data Set\\pyVHR\\ECG_FITNESS\\00\\01\\c920-1-R.txt")
g = np.loadtxt("E:\\Data Set\\pyVHR\\ECG_FITNESS\\00\\01\\c920-1-G.txt")
b = np.loadtxt("E:\\Data Set\\pyVHR\\ECG_FITNESS\\00\\01\\c920-1-B.txt")

inx = np.loadtxt("E:\\Data Set\\pyVHR\\ECG_FITNESS\\00\\01\\c920-1-patch_index.txt")
x = np.loadtxt("E:\\Data Set\\pyVHR\\ECG_FITNESS\\00\\01\\c920-1-X_Motion.txt")
y = np.loadtxt("E:\\Data Set\\pyVHR\\ECG_FITNESS\\00\\01\\c920-1-Y_Motion.txt")

rgb = np.array([[r,g,b]]).transpose((2, 0, 1))
mm = np.array([inx, x, y], np.int).transpose((1, 2, 0))

In [ ]:
dataset_name = 'ecg_fitness'                              # 数据集名称
video_DIR = r"E:\Data Set\pyVHR\ECG_FITNESS"           # 视频数据目录的路径
BVP_DIR = r"E:\Data Set\pyVHR\ECG_FITNESS"             # 相应的BVP数据目录的路径

seconds = 0                # 处理视频的时长
landmarks = extraction.MagicLandmarks.cheek_left_top + extraction.MagicLandmarks.cheek_right_top   # 选择Landmark点

sig_extractor = extraction.SignalProcessing()  # 创建一个可选用不同方法的用于提取信号的类
sig_extractor.set_skin_extractor(extraction.SkinExtractionConvexHull())  # 设置提取人脸皮肤的方法，ConvexHull法
sig_extractor.set_landmarks(landmarks)
sig_extractor.set_visualize_skin_and_landmarks(visualize_skin=True, visualize_landmarks=True,
                                               visualize_landmarks_number=False, visualize_patch=True)  # 可视化参数设置
sig_extractor.set_square_patches_side(10.0)  # 设置用于patches提取的矩形框尺寸


dataset = datasets.datasetFactory(dataset_name, videodataDIR=video_DIR, BVPdataDIR=BVP_DIR)  # 读取对应数据集的类方法
count = dataset.numVideos
print('All Videos: ', count)
start_time0 = time.time()

for index in range(count):
    fname = dataset.getSigFilename(index)                        # 读取索引为video_idx的视频，这里的范围与上述的all_video有关
    videoFileName = dataset.getVideoFilename(index)
    fps = int(extraction.get_fps(videoFileName))
    sig_extractor.set_total_frames(seconds * fps)                 # 得到需要处理的视频的帧数，seconds=0处理所有帧
    print(index, ': ', videoFileName)
    start_time1 = time.time()
    raw_Lab, _ = sig_extractor.extract_holistic(videoFileName)                      # Holistic方法

    visualize_coll = sig_extractor.get_visualize_skin()           # patches可视化图像信号
    skin_area = []
    for skin in visualize_coll:
        matrix_one = np.ones((skin.shape[0], skin.shape[1]))
        matrix_and = np.logical_and(matrix_one, skin[:,:,0])
        skin_area.append(sum(sum(matrix_and)))
    skin_area = np.array(skin_area)

    # 将提取到的Lab信息与运动信息分别存储
    txt_L_file = videoFileName[:29] + ' Signal' + videoFileName[29:-4] + '-Lab_L.txt'
    txt_a_file = videoFileName[:29] + ' Signal' + videoFileName[29:-4] + '-Lab_a.txt'
    txt_b_file = videoFileName[:29] + ' Signal' + videoFileName[29:-4] + '-Lab_b.txt'
    np.savetxt(txt_L_file, raw_Lab[:, :, 0])
    np.savetxt(txt_a_file, raw_Lab[:, :, 1])
    np.savetxt(txt_b_file, raw_Lab[:, :, 2])

    end_time1 = time.time()
    print('Get RGB signal time used :', end_time1 - start_time1)
    print(' ')

end_time0 = time.time()
print('All time used :', end_time0 - start_time0)


In [ ]:
dataset_name = 'ecg_fitness'                              # 数据集名称
video_DIR = r"D:\Data_Set\pyVHR\ECG_FITNESS Signal"         # 视频数据目录的路径
BVP_DIR = r"D:\Data_Set\pyVHR\ECG_FITNESS Signal"             # 相应的BVP数据目录的路径

seconds = 0                # 处理视频的时长
landmarks = extraction.MagicLandmarks.cheek_left_top + extraction.MagicLandmarks.cheek_right_top   # 选择Landmark点

sig_extractor = extraction.SignalProcessing()  # 创建一个可选用不同方法的用于提取信号的类
sig_extractor.set_skin_extractor(extraction.SkinExtractionConvexHull())  # 设置提取人脸皮肤的方法，ConvexHull法
sig_extractor.set_landmarks(landmarks)
sig_extractor.set_visualize_skin_and_landmarks(visualize_skin=True, visualize_landmarks=True,
                                               visualize_landmarks_number=False, visualize_patch=True)  # 可视化参数设置
sig_extractor.set_square_patches_side(10.0)  # 设置用于patches提取的矩形框尺寸


dataset = datasets.datasetFactory(dataset_name, videodataDIR=video_DIR, BVPdataDIR=BVP_DIR)  # 读取对应数据集的类方法
count = dataset.numVideos
print('All Videos: ', count)
start_time0 = time.time()
hr_gt = []
for index in range(count):
    fname = dataset.getSigFilename(index)                        # 读取索引为video_idx的视频，这里的范围与上述的all_video有关
    try:
        sigGT = dataset.readSigfile(fname)            # 读取视频对应的BVP.gt数据
    except:
        continue
    test_bvp = sigGT.data
    bpmGT, timesGT = sigGT.getBPM(winsize=20)         # GT信息
    hr_gt.append(np.array(bpmGT))
    print(index, ' : ', fname)

end_time0 = time.time()
print('All time used :', end_time0 - start_time0)

In [ ]:
hr_60 = []
for hr in hr_gt:
    if len(hr) > 60:
        ti = np.linspace(0, len(hr)-1, num=60, dtype=np.int16)
        hr_60.append(hr[ti])
    else:
        hr_60.append(hr)




In [ ]:
hr_delta = []
hr_delta_mu = []
hr_delta_std = []
for hr in hr_60:
    delta = abs(hr[1:] - hr[:-1]) / 60
    hr_delta.append(delta)
    hr_delta_mu.append(np.mean(delta))
    hr_delta_std.append(np.std(delta))

In [ ]:
hr_mean = np.mean(hr_delta_mu)
hr_std = np.std(hr_delta_std)

In [ ]:
dataset_name = 'ecg_fitness'                              # 数据集名称
video_DIR = r"D:\Data_Set\pyVHR\ECG_FITNESS Signal"         # 视频数据目录的路径
BVP_DIR = r"D:\Data_Set\pyVHR\ECG_FITNESS Signal"             # 相应的BVP数据目录的路径

w_size = 20                # 每次处理视频的滑窗长度 ，w_size=视频时长，则timesGT的第一个值对应的bpmGT中的心率就是对应整个视频时长的平均心率
overlap = 1                # 相邻滑窗在时间上的重叠长度
video_idx = 7          # 待处理视频的索引
seconds = 0                # 处理视频的时长
scal = 1.0                 # 视频尺寸缩放比例
landmarks = extraction.MagicLandmarks.cheek_left_top \
            + extraction.MagicLandmarks.cheek_right_top   # 选择Landmark点

dataset = datasets.datasetFactory(dataset_name, videodataDIR=video_DIR, BVPdataDIR=BVP_DIR)  # 读取对应数据集的类方法
fname = dataset.getSigFilename(video_idx)     # 读取索引为video_idx的视频，这里的范围与上述的all_video有关
videoFileName = dataset.getVideoFilename(video_idx)

print('All Videos: ', dataset.numVideos)
print('Video processed name: ', videoFileName)
fig = go.Figure()

for index in range(dataset.numVideos):
    fname = dataset.getSigFilename(index)                        # 读取索引为video_idx的视频，这里的范围与上述的all_video有关
    try:
        sigGT = dataset.readSigfile(fname)            # 读取视频对应的BVP.gt数据
    except:
        continue
    videoFileName = dataset.getVideoFilename(index)
    X_data_file = videoFileName[:-4] + "-1-X_Motion.txt"
    Y_data_file = videoFileName[:-4] + "-1-Y_Motion.txt"
    Z_data_file = videoFileName[:-4] + "-1-Z_Motion.txt"

    x_data = np.loadtxt(X_data_file)
    y_data = np.loadtxt(Y_data_file)
    z_data = np.loadtxt(Z_data_file)
    x_name = videoFileName[-14:-9] + "-X"
    y_name = videoFileName[-14:-9] + "-Y"
    z_name = videoFileName[-14:-9] + "-Z"

    r_color = random.randint(1, 255)
    g_color = random.randint(1, 255)
    b_color = random.randint(1, 255)
    fig.add_trace(go.Scatter(x=np.arange(x_data.shape[-1]), y=x_data[0],
                             mode='lines', marker_color='rgba(' + str(r_color) + ', ' + str(g_color) + ', ' + str(b_color) + ', 1.0)', name=x_name))
    fig.add_trace(go.Scatter(x=np.arange(y_data.shape[-1]), y=y_data[0],
                             mode='lines', marker_color='rgba(' + str(r_color) + ', ' + str(g_color) + ', ' + str(b_color) + ', 1.0)', name=y_name))
    fig.add_trace(go.Scatter(x=np.arange(z_data.shape[-1]), y=z_data,
                             mode='lines', marker_color='rgba(' + str(r_color) + ', ' + str(g_color) + ', ' + str(b_color) + ', 1.0)', name=z_name))

fig.update_layout(title='Motion')
fig.show()


In [ ]:
dataset_name = 'ecg_fitness'                              # 数据集名称
video_DIR = r"D:\Data_Set\pyVHR\ECG_FITNESS Signal"         # 视频数据目录的路径
BVP_DIR = r"D:\Data_Set\pyVHR\ECG_FITNESS Signal"             # 相应的BVP数据目录的路径

w_size = 20                # 每次处理视频的滑窗长度 ，w_size=视频时长，则timesGT的第一个值对应的bpmGT中的心率就是对应整个视频时长的平均心率
overlap = 1                # 相邻滑窗在时间上的重叠长度
video_idx = 0          # 待处理视频的索引
seconds = 0                # 处理视频的时长
scal = 1.0                 # 视频尺寸缩放比例
landmarks = extraction.MagicLandmarks.cheek_left_top \
            + extraction.MagicLandmarks.cheek_right_top   # 选择Landmark点

dataset = datasets.datasetFactory(dataset_name, videodataDIR=video_DIR, BVPdataDIR=BVP_DIR)  # 读取对应数据集的类方法
fname = dataset.getSigFilename(video_idx)     # 读取索引为video_idx的视频，这里的范围与上述的all_video有关
videoFileName = dataset.getVideoFilename(video_idx)

print('All Videos: ', dataset.numVideos)
print('Video processed name: ', videoFileName)
x = []
y = []
z = []
for index in range(dataset.numVideos):
    fname = dataset.getSigFilename(index)                        # 读取索引为video_idx的视频，这里的范围与上述的all_video有关
    try:
        sigGT = dataset.readSigfile(fname)            # 读取视频对应的BVP.gt数据
    except:
        continue
    videoFileName = dataset.getVideoFilename(index)
    X_data_file = videoFileName[:-4] + "-1-X_Motion.txt"
    Y_data_file = videoFileName[:-4] + "-1-Y_Motion.txt"
    Z_data_file = videoFileName[:-4] + "-1-Z_Motion.txt"

    x_data = np.loadtxt(X_data_file)
    y_data = np.loadtxt(Y_data_file)
    z_data = np.loadtxt(Z_data_file)
    x_name = videoFileName[-14:-9] + "-X"
    y_name = videoFileName[-14:-9] + "-Y"
    z_name = videoFileName[-14:-9] + "-Z"

    r_color = random.randint(1, 255)
    g_color = random.randint(1, 255)
    b_color = random.randint(1, 255)
    x.append((np.std(x_data[0])))
    y.append((np.std(y_data[0])))
    z.append((np.std(z_data)))

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(x)), y=x, mode='lines', name='x_std'))
fig.add_trace(go.Scatter(x=np.arange(len(y)), y=y, mode='lines', name='y_std'))
fig.add_trace(go.Scatter(x=np.arange(len(z)), y=z, mode='lines', name='z_std'))

fig.update_layout(title='Motion')
fig.show()

In [ ]:
dataset_name = 'ecg_fitness'                              # 数据集名称
video_DIR = r"D:\Data_Set\pyVHR\ECG_FITNESS Signal"         # 视频数据目录的路径
BVP_DIR = r"D:\Data_Set\pyVHR\ECG_FITNESS Signal"             # 相应的BVP数据目录的路径

w_size = 20                # 每次处理视频的滑窗长度 ，w_size=视频时长，则timesGT的第一个值对应的bpmGT中的心率就是对应整个视频时长的平均心率
overlap = 1                # 相邻滑窗在时间上的重叠长度
video_idx = 11              # 待处理视频的索引
seconds = 0                # 处理视频的时长
scal = 1.0                 # 视频尺寸缩放比例
landmarks = extraction.MagicLandmarks.cheek_left_top \
            + extraction.MagicLandmarks.cheek_right_top   # 选择Landmark点

dataset = datasets.datasetFactory(dataset_name, videodataDIR=video_DIR, BVPdataDIR=BVP_DIR)  # 读取对应数据集的类方法
fname = dataset.getSigFilename(video_idx)     # 读取索引为video_idx的视频，这里的范围与上述的all_video有关
videoFileName = dataset.getVideoFilename(video_idx)
fps = 30
print('All Videos: ', dataset.numVideos)
print('Video processed name: ', videoFileName)

index_data_file = videoFileName[:-4] + "-1-patch_index.txt"
X_data_file = videoFileName[:-4] + "-1-X_Motion.txt"
Y_data_file = videoFileName[:-4] + "-1-Y_Motion.txt"
Z_data_file = videoFileName[:-4] + "-1-Z_Motion.txt"
inx_data = np.loadtxt(index_data_file)
x_data = np.loadtxt(X_data_file)
y_data = np.loadtxt(Y_data_file)
z_data = np.loadtxt(Z_data_file)

motion_sig = np.array([inx_data, x_data, y_data], np.int32).transpose((1, 2, 0))
speed_x, speed_y, acc_x, acc_y = speed_acc_sig(motion_sig, fps)  # 数组形式速度、加速度信号
speed_x_sum, speed_y_sum, acc_x_sum, acc_y_sum, motion_sum = sum_speed_acc_sig(speed_x, speed_y, acc_x, acc_y, motion_sig)  # patches速度加速度求和
motion_z = z_data
speed_z_sum = np.array([np.zeros_like(motion_z)])
speed_z_sum[0, 1:] = np.array((motion_z[1:] - motion_z[0:-1]) * fps)
speed_sum = np.vstack([speed_x_sum, speed_y_sum, speed_z_sum])
speed_sum = np.expand_dims(speed_sum, axis = 2).transpose((1, 2, 0))

win_speed, times_ES = extraction.sig_windowing(speed_sum, 10, stride=1, fps=fps)
x_mean = []
y_mean = []
z_mean = []
for sig in win_speed:
    sig = np.squeeze(sig)
    x_mean.append(np.mean(abs(sig[0])))
    y_mean.append(np.mean(abs(sig[1])))
    z_mean.append(np.mean(abs(sig[2])))


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(x_mean)), y=x_mean, mode='lines', name='x_std'))
fig.add_trace(go.Scatter(x=np.arange(len(y_mean)), y=y_mean, mode='lines', name='y_std'))
fig.add_trace(go.Scatter(x=np.arange(len(z_mean)), y=z_mean, mode='lines', name='z_std'))

fig.update_layout(title='Speed-' + str(videoFileName[-11:-9]))
fig.show()